# Import libs

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from catboost import Pool, CatBoostRegressor
from scipy.signal import savgol_filter
from sklearn.model_selection import train_test_split

import os, sys
sys.path.append("../..")

import warnings
warnings.filterwarnings("ignore")

# Import Data

#### define paths

In [2]:
path_clusters_data = "../../data/clusters.parquet"

#### load datasets

In [3]:
df_clusters = pd.read_parquet(path_clusters_data)

In [4]:
df_clusters = df_clusters.drop(axis=1, columns=['int_index'])

In [5]:
df_clusters['real_weight_3_month_ago'] = df_clusters['real_weight'].shift(3)

In [6]:
df_clusters['real_weight_2_month_ago'] = df_clusters['real_weight'].shift(2)

In [7]:
df_clusters['real_weight_1_month_ago'] = df_clusters['real_weight'].shift(1)

In [8]:
df_clusters

,HASH,real_weight,real_wagon_count,price_urals_usd,rub-usd,rub-uan,density,k_density_before,k_density_after,cluster,real_weight_3_month_ago,real_weight_2_month_ago,real_weight_1_month_ago
2020-03-01,0,17490.0,249.0,29.51,73,10,70.240964,-0.038446,-0.047423,1,NaN,NaN,NaN
2020-04-01,0,16559.0,237.0,16.61,74,10,69.869198,-0.038446,-0.047423,1,NaN,NaN,17490.0
2020-05-01,0,5443.0,78.0,30.65,72,10,69.782051,-0.038446,-0.047423,1,NaN,17490.0,16559.0
2020-06-01,0,10975.0,157.0,42.36,69,9,69.904459,-0.038446,-0.047423,1,17490.0,16559.0,5443.0
2020-07-01,0,14957.0,214.0,44.28,71,10,69.892523,-0.038446,-0.047423,1,16559.0,5443.0,10975.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-01,75088,0.0,0.0,67.98,60,8,0.000000,0.000000,2.181818,1,0.0,0.0,0.0
2022-12-01,75088,0.0,0.0,52.23,65,9,0.000000,0.000000,2.181818,1,0.0,0.0,0.0
2023-01-01,75088,0.0,0.0,52.21,68,10,0.000000,0.000000,2.181818,1,0.0,0.0,0.0
2023-02-01,75088,0.0,0.0,51.91,72,10,0.000000,0.000000,2.181818,1,0.0,0.0,0.0


In [9]:
test = df_clusters[df_clusters.index.year == 2023].groupby(['HASH',df_clusters[df_clusters.index.year == 2023].index.year])[['real_weight']].median()

In [10]:
test.reset_index().drop(axis=1, columns='level_1')

,HASH,real_weight
0,0,18324.0
1,1,636.0
2,2,1327.0
3,3,867.0
4,4,334.0
...,...,...
75079,75084,0.0
75080,75085,0.0
75081,75086,0.0
75082,75087,0.0


In [11]:
df_clusters[df_clusters.HASH == 0]['mean_val'] = 18324

In [12]:
df_clusters['mean_last_10_month_weight'] = None

In [13]:
df_clusters['mean_last_10_month_wagon'] = None

In [14]:
dp = [0,0,0,0,0,0,0,0,0,0]
mem = []
for val in range(0, df_clusters.shape[0]):
    dp.pop(0)
    value = df_clusters['real_weight'][val]
    dp.append(value)
    mem.append(np.mean(dp))

In [15]:
df_clusters['mean_last_10_month_weight'] = mem

In [16]:
dp = [0,0,0,0,0,0,0,0,0,0]
mem = []
for val in range(0, df_clusters.shape[0]):
    dp.pop(0)
    value = df_clusters['real_wagon_count'][val]
    dp.append(value)
    mem.append(np.mean(dp))

In [17]:
df_clusters['mean_last_10_month_wagon'] = mem

In [18]:
df_clusters = df_clusters[df_clusters.index.year != 2020]

In [19]:
df_clusters.corr()

,HASH,real_weight,real_wagon_count,price_urals_usd,rub-usd,rub-uan,density,k_density_before,k_density_after,cluster,real_weight_3_month_ago,real_weight_2_month_ago,real_weight_1_month_ago,mean_last_10_month_weight,mean_last_10_month_wagon
HASH,1.000000e+00,-0.061916,-0.063812,-1.069441e-13,7.492039e-13,-1.126186e-13,-0.261499,4.296545e-02,2.284606e-02,3.738008e-02,-0.062969,-0.062534,-0.062284,-0.065239,-0.067148
real_weight,-6.191644e-02,1.000000,0.998855,-3.243156e-06,4.530350e-04,4.462677e-04,0.114739,-1.212237e-03,-3.853949e-03,1.575912e-04,0.940880,0.953042,0.971436,0.964777,0.963733
real_wagon_count,-6.381219e-02,0.998855,1.000000,-1.733794e-04,4.577889e-04,4.270685e-04,0.115180,-2.619296e-03,-2.590879e-03,2.441738e-04,0.940244,0.952249,0.970444,0.964234,0.964671
price_urals_usd,-1.069441e-13,-0.000003,-0.000173,1.000000e+00,8.916184e-02,1.735996e-01,0.006564,1.462176e-15,-3.065509e-15,3.020811e-15,0.000402,0.000595,0.000295,0.000464,0.000416
rub-usd,7.492039e-13,0.000453,0.000458,8.916184e-02,1.000000e+00,9.802044e-01,0.012746,1.750856e-14,7.412751e-15,1.584605e-14,0.000478,-0.000180,-0.000472,0.000392,0.000417
rub-uan,-1.126186e-13,0.000446,0.000427,1.735996e-01,9.802044e-01,1.000000e+00,0.014086,-8.564703e-16,-6.531053e-15,4.992715e-16,0.000649,0.000021,-0.000214,0.000507,0.000529
density,-2.614993e-01,0.114739,0.115180,6.563892e-03,1.274609e-02,1.408601e-02,1.000000,6.287438e-02,-6.829303e-02,2.036936e-02,0.106748,0.108007,0.110365,0.109117,0.109983
k_density_before,4.296545e-02,-0.001212,-0.002619,1.462176e-15,1.750856e-14,-8.564703e-16,0.062874,1.000000e+00,-5.838578e-02,7.411117e-01,-0.003119,-0.002450,-0.001831,-0.004227,-0.005543
k_density_after,2.284606e-02,-0.003854,-0.002591,-3.065509e-15,7.412751e-15,-6.531053e-15,-0.068293,-5.838578e-02,1.000000e+00,-1.797615e-01,-0.005666,-0.005090,-0.004516,-0.006002,-0.004364
cluster,3.738008e-02,0.000158,0.000244,3.020811e-15,1.584605e-14,4.992715e-16,0.020369,7.411117e-01,-1.797615e-01,1.000000e+00,-0.000945,-0.000559,-0.000192,-0.001686,-0.001846


In [20]:
df_clusters.head()

,HASH,real_weight,real_wagon_count,price_urals_usd,rub-usd,rub-uan,density,k_density_before,k_density_after,cluster,real_weight_3_month_ago,real_weight_2_month_ago,real_weight_1_month_ago,mean_last_10_month_weight,mean_last_10_month_wagon
2021-01-01,0,11189.0,155.0,54.89,74,11,72.187097,-0.038446,-0.047423,1,16098.0,17213.0,13516.0,12785.7,182.4
2021-02-01,0,8968.0,127.0,61.47,74,11,70.614173,-0.038446,-0.047423,1,17213.0,13516.0,11189.0,12026.6,171.4
2021-03-01,0,11867.0,171.0,64.29,74,11,69.397661,-0.038446,-0.047423,1,13516.0,11189.0,8968.0,12669.0,180.7
2021-04-01,0,14138.0,203.0,63.02,76,11,69.645320,-0.038446,-0.047423,1,11189.0,8968.0,11867.0,12985.3,185.3
2021-05-01,0,10179.0,146.0,67.26,74,11,69.719178,-0.038446,-0.047423,1,8968.0,11867.0,14138.0,12507.5,178.5


In [21]:
X = df_clusters.drop(axis=1, columns=['HASH','real_weight','real_wagon_count']).reset_index(drop=True)

# Apply t-SNE over dataset

In [22]:
# from sklearn.manifold import TSNE

In [23]:
# train_X = X.sample(250000)

In [24]:
# model_tsne = TSNE(n_components=3, learning_rate='auto',
#                  init='pca', perplexity=3).fit_transform(train_X)

# Train test split

In [25]:
y = df_clusters['real_wagon_count'].values

In [26]:
X = df_clusters.drop(axis=1, columns=["HASH",'real_weight','real_wagon_count']).reset_index(drop=True)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [38]:
# initialize Pool
train_pool = Pool(X_train, 
                  y_train,
                  cat_features=['cluster'])
test_pool = Pool(X_test, 
                 y_test,
                 cat_features=['cluster']) 

# Cpecify model params

In [56]:
params = {"iterations":500,"depth":4,"learning_rate":0.02,"loss_function":'MAE'}

In [57]:
model = CatBoostRegressor(**params)

In [58]:
model.fit(train_pool, plot=True, eval_set=test_pool, use_best_model=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 12.2309518	test: 12.8438448	best: 12.8438448 (0)	total: 674ms	remaining: 5m 36s
1:	learn: 12.2169447	test: 12.8298158	best: 12.8298158 (1)	total: 1.34s	remaining: 5m 34s
2:	learn: 12.1934675	test: 12.8063206	best: 12.8063206 (2)	total: 2.06s	remaining: 5m 41s
3:	learn: 12.1312951	test: 12.7437149	best: 12.7437149 (3)	total: 2.8s	remaining: 5m 46s
4:	learn: 12.0999786	test: 12.7122254	best: 12.7122254 (4)	total: 3.48s	remaining: 5m 44s
5:	learn: 12.0775805	test: 12.6897736	best: 12.6897736 (5)	total: 4.09s	remaining: 5m 36s
6:	learn: 12.0650237	test: 12.6771872	best: 12.6771872 (6)	total: 4.62s	remaining: 5m 25s
7:	learn: 12.0529199	test: 12.6650571	best: 12.6650571 (7)	total: 5.27s	remaining: 5m 23s
8:	learn: 12.0198154	test: 12.6319084	best: 12.6319084 (8)	total: 5.84s	remaining: 5m 18s
9:	learn: 11.9954159	test: 12.6074185	best: 12.6074185 (9)	total: 6.34s	remaining: 5m 10s
10:	learn: 11.9814988	test: 12.5934821	best: 12.5934821 (10)	total: 6.92s	remaining: 5m 7s
11:	learn:

90:	learn: 9.4175217	test: 10.0067375	best: 10.0067375 (90)	total: 48.2s	remaining: 3m 36s
91:	learn: 9.3699353	test: 9.9586414	best: 9.9586414 (91)	total: 48.7s	remaining: 3m 36s
92:	learn: 9.3654956	test: 9.9541248	best: 9.9541248 (92)	total: 49.2s	remaining: 3m 35s
93:	learn: 9.3096748	test: 9.8982280	best: 9.8982280 (93)	total: 49.9s	remaining: 3m 35s
94:	learn: 9.3050184	test: 9.8934685	best: 9.8934685 (94)	total: 50.4s	remaining: 3m 34s
95:	learn: 9.2986293	test: 9.8869555	best: 9.8869555 (95)	total: 50.9s	remaining: 3m 34s
96:	learn: 9.2940801	test: 9.8823827	best: 9.8823827 (96)	total: 51.4s	remaining: 3m 33s
97:	learn: 9.2923219	test: 9.8806031	best: 9.8806031 (97)	total: 51.9s	remaining: 3m 33s
98:	learn: 9.2903339	test: 9.8786010	best: 9.8786010 (98)	total: 52.5s	remaining: 3m 32s
99:	learn: 9.2368232	test: 9.8249253	best: 9.8249253 (99)	total: 53s	remaining: 3m 31s
100:	learn: 9.2328568	test: 9.8209079	best: 9.8209079 (100)	total: 53.5s	remaining: 3m 31s
101:	learn: 9.22877

181:	learn: 7.4563073	test: 8.0251910	best: 8.0251910 (181)	total: 1m 33s	remaining: 2m 43s
182:	learn: 7.4137376	test: 7.9830050	best: 7.9830050 (182)	total: 1m 34s	remaining: 2m 43s
183:	learn: 7.3635850	test: 7.9330917	best: 7.9330917 (183)	total: 1m 34s	remaining: 2m 42s
184:	learn: 7.3152007	test: 7.8845266	best: 7.8845266 (184)	total: 1m 35s	remaining: 2m 42s
185:	learn: 7.2683387	test: 7.8374929	best: 7.8374929 (185)	total: 1m 35s	remaining: 2m 41s
186:	learn: 7.2650798	test: 7.8341429	best: 7.8341429 (186)	total: 1m 36s	remaining: 2m 41s
187:	learn: 7.2650509	test: 7.8341106	best: 7.8341106 (187)	total: 1m 36s	remaining: 2m 40s
188:	learn: 7.2222456	test: 7.7911258	best: 7.7911258 (188)	total: 1m 37s	remaining: 2m 40s
189:	learn: 7.1777113	test: 7.7468793	best: 7.7468793 (189)	total: 1m 37s	remaining: 2m 39s
190:	learn: 7.1746033	test: 7.7437043	best: 7.7437043 (190)	total: 1m 38s	remaining: 2m 39s
191:	learn: 7.1312374	test: 7.7006903	best: 7.7006903 (191)	total: 1m 38s	remain

271:	learn: 5.2160969	test: 5.8131370	best: 5.8131370 (271)	total: 2m 19s	remaining: 1m 56s
272:	learn: 5.2023574	test: 5.7984651	best: 5.7984651 (272)	total: 2m 19s	remaining: 1m 56s
273:	learn: 5.1873579	test: 5.7818932	best: 5.7818932 (273)	total: 2m 20s	remaining: 1m 55s
274:	learn: 5.1850852	test: 5.7794452	best: 5.7794452 (274)	total: 2m 20s	remaining: 1m 55s
275:	learn: 5.1818537	test: 5.7758666	best: 5.7758666 (275)	total: 2m 21s	remaining: 1m 54s
276:	learn: 5.1694207	test: 5.7621927	best: 5.7621927 (276)	total: 2m 21s	remaining: 1m 54s
277:	learn: 5.1691400	test: 5.7619193	best: 5.7619193 (277)	total: 2m 22s	remaining: 1m 53s
278:	learn: 5.1685721	test: 5.7613155	best: 5.7613155 (278)	total: 2m 22s	remaining: 1m 52s
279:	learn: 5.1566504	test: 5.7482294	best: 5.7482294 (279)	total: 2m 23s	remaining: 1m 52s
280:	learn: 5.1451515	test: 5.7356624	best: 5.7356624 (280)	total: 2m 23s	remaining: 1m 51s
281:	learn: 5.1448891	test: 5.7354061	best: 5.7354061 (281)	total: 2m 24s	remain

361:	learn: 5.0200282	test: 5.6003057	best: 5.6003057 (361)	total: 3m	remaining: 1m 8s
362:	learn: 5.0199340	test: 5.6002027	best: 5.6002027 (362)	total: 3m	remaining: 1m 8s
363:	learn: 5.0198222	test: 5.6000883	best: 5.6000883 (363)	total: 3m 1s	remaining: 1m 7s
364:	learn: 5.0197151	test: 5.5999793	best: 5.5999793 (364)	total: 3m 1s	remaining: 1m 7s
365:	learn: 5.0171833	test: 5.5976350	best: 5.5976350 (365)	total: 3m 2s	remaining: 1m 6s
366:	learn: 5.0170799	test: 5.5975295	best: 5.5975295 (366)	total: 3m 2s	remaining: 1m 6s
367:	learn: 5.0170353	test: 5.5974846	best: 5.5974846 (367)	total: 3m 3s	remaining: 1m 5s
368:	learn: 5.0125042	test: 5.5930321	best: 5.5930321 (368)	total: 3m 3s	remaining: 1m 5s
369:	learn: 5.0057017	test: 5.5850218	best: 5.5850218 (369)	total: 3m 4s	remaining: 1m 4s
370:	learn: 5.0041751	test: 5.5834278	best: 5.5834278 (370)	total: 3m 4s	remaining: 1m 4s
371:	learn: 5.0039680	test: 5.5832289	best: 5.5832289 (371)	total: 3m 5s	remaining: 1m 3s
372:	learn: 5.00

452:	learn: 4.9421301	test: 5.5154763	best: 5.5154763 (452)	total: 3m 42s	remaining: 23.1s
453:	learn: 4.9420981	test: 5.5154374	best: 5.5154374 (453)	total: 3m 43s	remaining: 22.6s
454:	learn: 4.9412340	test: 5.5145431	best: 5.5145431 (454)	total: 3m 43s	remaining: 22.1s
455:	learn: 4.9411787	test: 5.5144806	best: 5.5144806 (455)	total: 3m 44s	remaining: 21.6s
456:	learn: 4.9362235	test: 5.5085669	best: 5.5085669 (456)	total: 3m 44s	remaining: 21.1s
457:	learn: 4.9361606	test: 5.5085030	best: 5.5085030 (457)	total: 3m 44s	remaining: 20.6s
458:	learn: 4.9361337	test: 5.5084703	best: 5.5084703 (458)	total: 3m 45s	remaining: 20.1s
459:	learn: 4.9360913	test: 5.5084282	best: 5.5084282 (459)	total: 3m 45s	remaining: 19.6s
460:	learn: 4.9360415	test: 5.5083700	best: 5.5083700 (460)	total: 3m 46s	remaining: 19.1s
461:	learn: 4.9359771	test: 5.5083048	best: 5.5083048 (461)	total: 3m 46s	remaining: 18.6s
462:	learn: 4.9359292	test: 5.5082623	best: 5.5082623 (462)	total: 3m 47s	remaining: 18.2s